In [21]:
import requests
import os

base_url = 'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/'

dict_country = {
    1: "Alabama", 2: "Arizona", 3: "Arkansas", 4: "California", 
    5: "Colorado", 6: "Connecticut", 7: "Delaware", 8: "Florida",
    9: "Georgia", 10: "Idaho", 11: "Illinois", 12: "Indiana", 
    13: "Iowa", 14: "Kansas", 15: "Kentucky", 16: "Louisiana", 
    17: "Maine", 18: "Maryland", 19: "Massachusetts", 20: "Michigan",
    21: "Minnesota", 22: "Mississippi", 23: "Missouri", 24: "Montana", 
    25: "Nebraska", 26: "Nevada", 27: "New Hampshire", 28: "New Jersey", 
    29: "New Mexico", 30: "New York", 31: "North Carolina", 
    32: "North Dakota", 33: "Ohio", 34: "Oklahoma", 35: "Oregon",
    36: "Pennsylvania", 37: "Rhode Island", 38: "South Carolina", 
    39: "South Dakota", 40: "Tennessee", 41: "Texas", 42: "Utah", 
    43: "Vermont", 44: "Virginia", 45: "Washington", 46: "West Virginia",
    47: "Wisconsin", 48: "Wyoming", 50: "Alaska"
}

dict_stats = {
    'tavg' : 'Average Temperature',
    'tmax' : 'Max Temperature',
    'tmin' : 'Min Temperature',
    'pcp' : 'Precipitation',
    'cdd' : 'Cooling Degree Days',
    'hdd' : 'Heating Degree Days',
    'pdsi' : 'Palmer Drought Severity Index - PDSI',
    'phdi' : 'Palmer Hydrological Drought Index - PHDI',
    'pmdi' : 'Palmer Modified Drought Index - PMDI',
    'zndx' : 'Palmer Z-Index'
}

def download_weather():
    base_folder = 'weather_data'
    os.makedirs(base_folder, exist_ok = True)
    for country, country_name in dict_country.items():
        state_folder = os.path.join(base_folder, country_name)
        os.makedirs(state_folder, exist_ok=True)
        
        for stat, stat_name in dict_stats.items():          
            url = base_url + f'{country}/{stat}/1/0/1997-2022.csv?base_prd=true&amp;begbaseyear=1901&amp;endbaseyear=2000'
            response = requests.get(url)
            file_name = f'{stat}.csv'
            file_path = os.path.join(state_folder, file_name)
            with open(file_path, "wb") as file:
                file.write(response.content)         

# https://www.ncei.noaa.gov/pub/data/cirs/climdiv/state-readme.txt - State values

In [23]:
download_weather()

In [47]:
import pandas as pd

def collate_state():
    base_folder = "weather_data"
    output_folder = "collated_data"
    os.makedirs(output_folder, exist_ok=True)
    for state_folder in os.listdir(base_folder):
        state_data = []
        if state_folder == 'Alaska':
            continue
        state_path = os.path.join(base_folder, state_folder)
        for stat_file in os.listdir(state_path):
            stat_path = os.path.join(state_path, stat_file)
            if stat_file.endswith(".csv"):
                stat_name = os.path.splitext(stat_file)[0] 
                if stat_name in ['pmdi', 'pdsi', 'zndx', 'phdi']:
                    df = pd.read_csv(stat_path, skiprows=3, usecols=["Date", "Value"])
                else:
                    df = pd.read_csv(stat_path, skiprows=4, usecols=["Date", "Value"])
                df.rename(columns={"Value": stat_name}, inplace=True)
                state_data.append(df)
        merged_df = state_data[0]
        for df in state_data[1:]:
            merged_df = pd.merge(merged_df, df, on="Date", how="outer")
        output_path = os.path.join(output_folder, f"{state_folder}.csv")
        merged_df.to_csv(output_path, index=False)

# We are ignoring Alaska for this project

In [49]:
collate_state()

In [82]:
# State-Wise Energy Consumption - Yearly
# https://www.eia.gov/state/seds/seds-data-complete.php?sid=MI#Keystatisticsrankings

state_name_dict = {
        "AK": "Alaska", "AL": "Alabama", "AR": "Arkansas", "AZ": "Arizona", 
        "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", 
        "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho", 
        "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", 
        "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland", 
        "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", 
        "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", 
        "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York", 
        "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma", 
        "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", 
        "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah", 
        "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia", 
        "WI": "Wisconsin", "WY": "Wyoming"
    }

def process_excel():
    output_folder = 'power_consumption'
    os.makedirs(output_folder, exist_ok = True)
    state_data_combined = {}
    sheet_dict = {
        'use_tot_realgdp.xlsx': ['Real GDP', 'Energy consumption per real GDP'],
        'use_tot_sector.xlsx': ['Residential sector','Commercial sector', 'Industrial sector', 'Transportation sector', 'Total consumption']
    }
    for file_path, sheet_names in sheet_dict.items():
        for sheet_name in sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=2)
            # df.columns = [col.strip() for col in df.columns]
            states = df.iloc[:, 0]
            years = df.columns[1:]
            for state in states.unique():
                if state in ['AK', 'US', 'HI', 'DC']:
                    continue
                if pd.isna(state):
                    continue
                state_data = df[df.iloc[:, 0] == state]
                state_df = state_data.iloc[:, 1:].T
                state_df.columns = [sheet_name]
                state_df.index = years
                state_df.index.name = "Year"
                state_df = state_df.loc[1997:2022]
                if state not in state_data_combined:
                    state_data_combined[state] = state_df
                else:
                    state_data_combined[state] = pd.concat([state_data_combined[state], state_df], axis=1)
    for state, data in state_data_combined.items():
        state_name = state_name_dict.get(state, state)
        state_file_path = os.path.join(output_folder, f"{state_name}.csv")
        data.to_csv(state_file_path)

In [84]:
process_excel()

In [146]:
# Wildfire Data
# https://www.nifc.gov/fire-information/statistics/wildfires
from bs4 import BeautifulSoup

def get_wildfire():
    url = 'https://www.nifc.gov/fire-information/statistics/wildfires'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    tr = soup.find('div').find('tbody').find_all('tr')[2:-1]
    columns = []
    for td in tr[0].find_all('td'):
        columns.append(td.text)
    fire = pd.DataFrame(columns=columns)
    for _ in tr[2:]: # starting from 2022
        row_append = []
        for td in _.find_all('td'):
            row_append.append(int(td.text.strip().replace('*', '').replace(',','')))
        fire.loc[len(fire)] = row_append
    fire.set_index('Year', inplace= True)
    fire.index = pd.to_numeric(fire.index, errors='coerce')
    fire = fire[fire.index >= 1997]
    fire = fire[fire.index <= 2022]
    fire = fire[::-1]
    fire.to_csv('wildfires.csv')

In [148]:
get_wildfire()

In [204]:
# Monthly and Annual Breakdown of Power Data - Breakdown of Source of Consumption
# https://www.eia.gov/totalenergy/data/browser/index.php?tbl=T01.01#/?f=M&start=199701&end=202407&charted=6-7-14-1-2-3-4-8-11-12-13

def get_monthly_power():
    output_file = 'power_consumptions_monthly.csv'
    
    df = pd.read_excel('Table_1.1_Primary_Energy_Overview.xlsx', sheet_name='Monthly Data', skiprows=10)
    df = df.drop(index=0)
    df['Month'] = pd.to_datetime(df['Month'], errors='coerce')
    df['Month'] = df['Month'].dt.strftime('%Y%m').astype(int)
    df = df[(df['Month'] >= 199701) & (df['Month'] <= 202212)]
    df.set_index('Month', inplace=True)
    df = df.iloc[:, -4:]
    df.to_csv(output_file)

def get_yearly_power():
    output_file = 'power_consumptions_yearly.csv'
    df = pd.read_excel('Table_1.1_Primary_Energy_Overview.xlsx', sheet_name='Annual Data', skiprows=10)
    df = df.drop(index=0)
    
    df = df.iloc[:, [0, -4, -3, -2, -1]]
    df.columns = ['Year'] + list(df.columns[1:])
    df['Year'] = df['Year'].astype(int)
    df = df[(df['Year'] >= 1997) & (df['Year'] <= 2022)]
    df.set_index('Year', inplace=True)
    df.to_csv(output_file)

In [206]:
get_monthly_power()
get_yearly_power()